# Developement of optimization algorithm

The past attemps of improving the computer win percentage beyound the results fitting random data sets were not successful. When using the improved computer the main problem was that the computer mainly fit the computer model but that it did not improve in more general cases.
There are not enough computer versus human to use them efficiently but possible the same problem exist also there.

Thus a new method is needed. 
The steps of it are

0) start with random model or with result from other fits? 
1) create a new computer model rather random first
later gradient methods can be used to combine past models
2) play sufficently often against existing models that win percentages are reliable enough. 

Iterate step 1 to 2 for some time, test against humans sometimes.

Do the models need a random component? Can be set easily to 0, with a paramter, the oppiste is not true, thus included.  

Steps needed:
    1) New random model which follows the same principle than the not random models need to be build.
    2) Full model or step wise?

Full means best option is predicted from a single model. 
Otherwise only each step is predicted, still needs information on the previous steps in the process. 

I decide for step wise. 

1) Is whether to take open card. Is binary, thus logistic function used. Parameters, for now minumum amount no chance to consider vanishing card for now:
    value_open_card, sum_own, own_n_closed, sum_other, other_n_closed
2) (Is not always applied) Is whether to take previously closed card: Is binary, thus logistic function used. Parameters, for now minumum amount no chance to consider vanishing card for now:
    value_closed_card, sum_own, own_n_closed, sum_other, other_n_closed
3) which own card to give if not random trun which is random, is numeric, options are the cards available and closed cards, depends on available cards, in particular the largest value, when that is used exclusively of the open one, it gets a binary problem, again logistic, parameters: Value of_exchangable_card, max_own, sum_own, own_n_closed, sum_other, other_n_closed
 
 delta_score (own-other current score) would make sense for all but more difficult to add, keep it simple for now 

Means that random is not anymore random here but more clever but fine as start.

In [1]:
import numpy as np
import random as random
import scipy
#using simpleguitk for display, is not needed for computer game
import simpleguitk as simplegui
import time
#new ones
import pandas as pd
#ml methods
from xgboost import XGBRegressor
from xgboost import XGBClassifier
#logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import os
#skyjo game classes and functions 
from skyjo_functions4 import *
#own functions for machine learning
from ml_functions2 import *
#for plotting 
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

#for confidence intervalls
from scipy.stats import beta
#for splitting
from sklearn.model_selection import train_test_split
#confusing matrix
from sklearn.metrics import confusion_matrix

Now I work on the fitting algorithm. The method is gradient fit, but it is not against a constant truth asusual but a changing model. At the beginning it is just, the all 0 model. As the first step all other parameters are separatedly changed by an arbitrary step. 

In [2]:
#means changes within the function do not have an effect outside of it , copy it to the top functions does not change anything 
def first_gradient_step(open_steps,discard_steps,value_steps,realizations):
    n_it=realizations
    results=np.zeros((40,19))
    for j in range(results.shape[1]):
        print(f"doing case {j}")
        level20_open_variable=np.zeros((6))
        level20_discard_variable=np.zeros((6))
        level20_value_variable=np.zeros((7))
        level21_open_variable=np.zeros((6))
        level21_discard_variable=np.zeros((6))
        level21_value_variable=np.zeros((7))
        if j<6:
            level21_open_variable[j]=open_steps[j]
        elif j<12:
            level21_discard_variable[j-6]=discard_steps[j-6]
        else:
            level21_value_variable[j-12]=value_steps[j-12]    
        print(level21_open_variable,level21_discard_variable,level21_value_variable)
        results[0:6,j]=level20_open_variable
        results[6:12,j]=level20_discard_variable
        results[12:19,j]=level20_value_variable    
        results[19:25,j]=level21_open_variable
        results[25:31,j]=level21_discard_variable
        results[31:38,j]=level21_value_variable         
        win20=0
        start_time=time.time()
        for i in range(n_it):
            names=['alpha','beta']
            nature=['computer','computer']
            levels=[20,21]
            winner=skyjo_game(names,nature,levels,0,True,False,level20_open_variable=level20_open_variable,level21_open_variable=level21_open_variable,level20_discard_variable=level20_discard_variable,level21_discard_variable=level21_discard_variable,level20_value_variable=level20_value_variable,level21_value_variable=level21_value_variable)
            if winner[0]==1:
                win20+=1
        results[38,j]=n_it
        results[39,j]=win20
        stop_time=time.time()
        print(f"{n_it} games need {np.round(stop_time-start_time,3)} seconds")
        print(f"level 20 won to {np.round(win20*100/n_it,1)} %")
    return results     

In [ ]:
open_step=[0.05,0.05,0.05,0.05,0.05,0.05]
discard_step=[0.05,0.05,0.05,0.05,0.05,0.05]
value_step=[0.05,0.05,0.05,0.05,0.05,0.05,0.05]
#does not recreated levls good 
result=first_gradient_step(open_step,discard_step,value_step,400)
np.savetxt("gradient_level20_v5.txt",result)

doing case 0
[0.05 0.   0.   0.   0.   0.  ] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 20.04 seconds
level 20 won to 62.0 %
doing case 1
[0.   0.05 0.   0.   0.   0.  ] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 21.75 seconds
level 20 won to 65.2 %
doing case 2
[0.   0.   0.05 0.   0.   0.  ] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 23.911 seconds
level 20 won to 57.8 %
doing case 3
[0.   0.   0.   0.05 0.   0.  ] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 25.4 seconds
level 20 won to 64.5 %
doing case 4
[0.   0.   0.   0.   0.05 0.  ] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 25.037 seconds
level 20 won to 58.2 %
doing case 5
[0.   0.   0.   0.   0.   0.05] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0.]
400 games need 24.431 seconds
level 20 won to 57.5 %
doing case 6
[0. 0. 0. 0. 0. 0.] [0.05 0.   0.   0.   0.   0.  ] [0. 0. 0. 0. 0. 0. 0.]
400 games need 17.932 seconds
level 20 won to 82.0 %
doing case 7
[0. 

Is now working. The win percentage changes now, see below. 

In [19]:
print(result[39]/result[38]*100)

[59.25 65.5  56.5  59.   58.   61.75 86.25 67.75 86.75 83.5  81.75 86.25
 10.75 10.25  7.25  7.5  13.   14.5  12.25]
